# Multiprocessing/ threading

In [25]:
import numpy as np
import requests
import threading
import concurrent.futures
from tqdm.notebook import tqdm
from time import sleep
import time
import pylab
import os
import matplotlib.pyplot as plt
from matplotlib import style
#speed things up
import numba
from numba import njit
from numba import jit
from numba import prange
from numba_progress import ProgressBar
from scipy.ndimage import convolve, generate_binary_structure
from timeit import default_timer as timer
#style
plt.style.use(['science','notebook','grid'])

In [6]:
start = time.perf_counter()



def do_something () :
    print('Sleeping 1 second...')
    sleep(1)
    print('Done Sleeping...')




do_something()
do_something()
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second (s) ')

Sleeping 1 second...
Done Sleeping...
Sleeping 1 second...
Done Sleeping...
Finished in 2.0 second (s) 


In [8]:
#let's create the threads
start = time.perf_counter()
def do_something () :
    print('Sleeping 1 second...')
    sleep(1)
    print('Done Sleeping...')
finish = time.perf_counter()

t1 = threading.Thread(target=do_something)
t2 = threading.Thread(target=do_something)
#if we compile this it run immediately but nothin its printed out so..

In [9]:
t1.start()
t2.start()

Sleeping 1 second...
Sleeping 1 second...
Done Sleeping...Done Sleeping...



In [10]:
start = time.perf_counter()
def do_something () :
    print('Sleeping 1 second...')
    sleep(1)
    print('Done Sleeping...')
finish = time.perf_counter()

t1 = threading.Thread(target=do_something)
t2 = threading.Thread(target=do_something)

t1.start()
t2.start()
print(f'Finished in {round(finish-start, 2)} second (s) ') #it started both the threads and while threading our script run cuncorrently

Sleeping 1 second...
Sleeping 1 second...
Finished in 0.0 second (s) 
Done Sleeping...
Done Sleeping...


In [11]:
#in order to wait that our threads finish to run: (join)
start = time.perf_counter()
def do_something () :
    print('Sleeping 1 second...')
    sleep(1)
    print('Done Sleeping...')
finish = time.perf_counter()

t1 = threading.Thread(target=do_something)
t2 = threading.Thread(target=do_something)

t1.start()
t2.start()

t1.join()
t2.join()

print(f'Finished in {round(finish-start, 2)} second (s) ') #it started both the threads and while threading our script run cuncorrently

Sleeping 1 second...
Sleeping 1 second...
Done Sleeping...
Done Sleeping...
Finished in 0.0 second (s) 


In [14]:
#threads in a loop
start = time.perf_counter()
def do_something () :
    print('Sleeping 1 second...')
    sleep(1)
    print('Done Sleeping...')

    
threads=[]
for _ in range(10): 
    t = threading.Thread(target=do_something)  #join not allowed in loop
    t.start()
    threads.append(t)
    
for thread in threads:
    thread.join()
    
finish = time.perf_counter()

         
print(f'Finished in {round(finish-start, 2)} second (s) ') #it started both the threads and while threading our script run cuncorrently

Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Sleeping 1 second...
Done Sleeping...Done Sleeping...

Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Finished in 1.0 second (s) 


In [16]:
#arguments
start = time.perf_counter()
def do_something(seconds) :
    print(f'Sleeping {seconds} second(s)...')
    sleep(seconds)
    print('Done Sleeping...')

threads=[]
for _ in range(10): 
    t = threading.Thread(target=do_something, args=[1.5]) #arguments passed by a list
    t.start()
    threads.append(t)
    
for thread in threads:
    thread.join()
    
finish = time.perf_counter()

         
print(f'Finished in {round(finish-start, 2)} second (s) ')

Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Sleeping 1.5 second(s)...
Done Sleeping...Done Sleeping...

Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Finished in 1.51 second (s) 


In [20]:
#manual way to create threads (ThreadPool executor)
start = time.perf_counter()
def do_something(seconds) :
    print(f'Sleeping {seconds} second(s)...')
    sleep(seconds)
    return 'Done Sleeping...'

with concurrent.futures.ThreadPoolExecutor() as executor:
    #execute a function one at time ---> submit method (return a future objects that allow us to grab the eventual return or check the state of our function)
    f1 = executor.submit(do_something, 1)  #argument 1 second
    f2 = executor.submit(do_something, 1)
    print(f1.result())
    print(f2.result())
    
    
    

# threads=[]
# for _ in range(10): 
#     t = threading.Thread(target=do_something, args=[1.5]) #arguments passed by a list
#     t.start()
#     threads.append(t)

# for thread in threads:
#     thread.join()
finish = time.perf_counter()


print(f'Finished in {round(finish-start, 2)} second (s) ')






Sleeping 1 second(s)...
Sleeping 1 second(s)...
Done Sleeping...
Done Sleeping...
Finished in 1.01 second (s) 


In [22]:
#with loop
start = time.perf_counter()
def do_something(seconds) :
    print(f'Sleeping {seconds} second(s)...')
    sleep(seconds)
    return f'Done Sleeping...{seconds}'

with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = [executor.submit(do_something, sec) for sec in secs]
    for f in concurrent.futures.as_completed(results):
        print(f.result())
    

# threads=[]
# for _ in range(10): 
#     t = threading.Thread(target=do_something, args=[1.5]) #arguments passed by a list
#     t.start()
#     threads.append(t)

# for thread in threads:
#     thread.join()
finish = time.perf_counter()


print(f'Finished in {round(finish-start, 2)} second (s) ')
# finishe in diff order cause  as_complethed method printed out in the order that are completed

Sleeping 5 second(s)...
Sleeping 4 second(s)...
Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
Done Sleeping...1
Done Sleeping...2
Done Sleeping...3
Done Sleeping...4
Done Sleeping...5
Finished in 5.01 second (s) 


In [23]:
#map method
start = time.perf_counter()
def do_something(seconds) :
    print(f'Sleeping {seconds} second(s)...')
    sleep(seconds)
    return f'Done Sleeping...{seconds}'

with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = executor.map(do_something, secs) #map runs the function for every value in secs 
                                                #instead of the previous that returned a future objet map will return the result
                                                #in the order they were started
    for result in results:
        print(result)

finish = time.perf_counter()


print(f'Finished in {round(finish-start, 2)} second (s) ')



Sleeping 5 second(s)...Sleeping 4 second(s)...
Sleeping 3 second(s)...

Sleeping 2 second(s)...
Sleeping 1 second(s)...
Done Sleeping...5
Done Sleeping...4
Done Sleeping...3
Done Sleeping...2
Done Sleeping...1
Finished in 5.01 second (s) 


In [24]:
#map method
start = time.perf_counter()
def do_something(seconds) :
    print(f'Sleeping {seconds} second(s)...')
    sleep(seconds)
    return f'Done Sleeping...{seconds}'

with concurrent.futures.ThreadPoolExecutor() as executor:
    secs = [5,4,3,2,1]
    results = executor.map(do_something, secs) #map runs the function for every value in secs 
                                                #instead of the previous that returned a future objet map will return the result
                                                #in the order they were started
    # for result in results:
    #     print(result)

finish = time.perf_counter()


print(f'Finished in {round(finish-start, 2)} second (s) ')   
#it still waited for the threads to complete

Sleeping 5 second(s)...Sleeping 4 second(s)...

Sleeping 3 second(s)...
Sleeping 2 second(s)...
Sleeping 1 second(s)...
Finished in 5.01 second (s) 


In [26]:
#real world example
img_urls = [
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1530224264768-7ff8c1789d79',
    'https://images.unsplash.com/photo-1564135624576-c5c88640f235',
    'https://images.unsplash.com/photo-1541698444083-023c97d3f4b6',
    'https://images.unsplash.com/photo-1522364723953-452d3431c267',
    'https://images.unsplash.com/photo-1513938709626-033611b8cc03',
    'https://images.unsplash.com/photo-1507143550189-fed454f93097',
    'https://images.unsplash.com/photo-1493976040374-85c8e12f0c0e',
    'https://images.unsplash.com/photo-1504198453319-5ce911bafcde',
    'https://images.unsplash.com/photo-1530122037265-a5f1f91d3b99',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1549692520-acc6669e2f0c'
]

def download_image(img_url):                     #normally one does a for loop for img_url in img_urls and does the followin but...
    img_bytes = requests.get(img_url).content    #it would download these one at a time
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        print(f'{img_name} was downloaded...')
        
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(download_image, img_urls)


t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

t1 = time.perf_counter()

### Let's try threading for ising model   

In [27]:
f_ext=0

In [28]:
#funzione che calcola che tiene conto di PBC per un generico reticolo rettangolare (nlatt_x,nlatt_y)
def boundaries_cond(nlatt_x,nlatt_y):
    nlatt_x = int(nlatt_x)
    nlatt_y = int(nlatt_y)
    if (nlatt_x == nlatt_y):
        npp = np.zeros(nlatt_x).astype(np.int64)
        nmm = np.zeros(nlatt_x).astype(np.int64)
        
        for i in range(nlatt_x):
            npp[i] = i+1
            nmm[i] = i-1
        npp[nlatt_x - 1] = 0
        nmm[0] = nlatt_x - 1
        return npp,nmm

    else:
        nppx = np.zeros(nlatt_x).astype(np.int64)
        nmmx = np.zeros(nlatt_x).astype(np.int64)
        nppy = np.zeros(nlatt_y).astype(np.int64)
        nmmy = np.zeros(nlatt_y).astype(np.int64)
        for i in range(nlatt_x):
            nppx[i] = i + 1
            nmmx[i] = i - 1
        nppx[nlatt_x - 1] = 0
        nmmx[0]= nlatt_x - 1

        for i in range(nlatt_y):
            nppy[i] = i+1    
            nmmy[i] = i-1
        nppy[nlatt_y - 1] = 0
        nmmy[0]  = nlatt_y - 1

        return nppx,nmmx,nppy,nmmy 

In [29]:
@jit(parallel=True)
def init(flag, Nx, Ny):   # 0 parte a freddo , 1 a caldo 50 su e 50 giu, ----> aggiungere carica da file o partenze particolari
    field = np.zeros((Nx,Ny))
    if (flag == 0):      
        field[:][:] = 1
        return field
    elif (flag == 1):
        for i in prange (0,Nx):
            for j in prange(0,Ny):
                r = np.random.uniform(0,1)
                if (r<0.5):
                     field[i][j] = 1
                else:
                    field[i][j] = -1
        return field 

In [30]:
@njit("(f8[:,:],f8, i8[:], i8[:])", cache=True)
def metropolis(field, beta, npp, nmm):
    nvol=int(field.shape[0]*field.shape[1])
    Nx = field.shape[0]
    Ny = field.shape[1]
    for i in range(0,nvol):
        x = int(np.random.uniform(0,1) * Nx)    #python indicizza da 0 a n-1 per a[n]
        y = int(np.random.uniform(0,1) * Ny)
        
        xp = npp[x]
        xm = nmm[x]
        yp = npp[y]
        ym = nmm[y]
        
        f = beta * (field[x][yp] + field[x][ym] + field[xp][y] + field[xm][y] + f_ext)   #calcolo la 'forza' + eventuale campo esterno
        
        s_i = field[x][y]    #spin attuale
        
        p_rat = np.exp(-2 * f * s_i)
        r = np.random.uniform(0,1)     #test accettanza
        
        if(r < p_rat):
            field[x][y] = -1*s_i

In [31]:
@njit("f8(f8[:,:], i8[:], i8[:])")
def energy(field, npp, nmm):
    ene=0.0
    for x in range(0,field.shape[0]):
        for y in range(0,field.shape[1]):
            xp = npp[x]
            xm = nmm[x]
            yp = npp[y]
            ym = nmm[y]
            f = field[x][yp] + field[x][ym] + field[xp][y] + field[xm][y]
            ene = ene -  0.5*f*field[x,y]
            ene = ene - f_ext*field[x,y]
    ene = ene/(field.shape[0]*field.shape[1])
    return ene  

In [32]:
@njit("f8(f8[:,:], i8[:], i8[:])")
def magnetization(field, npp, nmm):
    magn=0.0
    for x in prange(0, field.shape[0]):
        for y in prange(0, field.shape[1]):
            magn += field[x,y]
    magn = magn/(field.shape[0]*field.shape[1])
    return magn

In [33]:
beta_exp = np.round(np.arange(0.3,0.55,0.002), decimals=3)
beta_exp_try = np.round(np.arange(0.3,0.31,0.001), decimals=3)
dim = np.arange(3,10,2)
dim_try =np.arange(10,30,10)
#for simplicity let's consider only a square like lattice
N=50
save_measure=25
measures=2000000
measures_try=10000

In [34]:
def sim(beta, measures, save_step, n_latt):
    npp, nmm = boundaries_cond(n_latt,n_latt)
    filename=f"thread_{n_latt}.txt"
    file = open(filename,"w")
    file.write("#betas\t\tEnergy\t\tMagn\n")
    for bs in beta:
        field = init(1, n_latt, n_latt)
        for meas in range(measures):
            metropolis(field, bs, npp, nmm)
            if(meas%save_measure==0):
                file.write("%f\t\t%f\t\t%f\n"%(bs, energy(field, npp, nmm), magnetization(field, npp, nmm)))
    file.close()

In [49]:
def sim_n(n_latt):
    for n in tqdm(n_latt):
        beta=np.round(np.arange(0.3,0.55,0.002))
        measures=1000
        save_step=1
        sim(beta, measures, save_step, n)
 

In [ ]:
sim_n(beta_exp, measures, save_measure, dim)

In [56]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(sim_n,np.arange(3,10,2))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]